In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import itertools

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 分类存储数据

In [2]:
import os

def mkdir_if_not_exist(path):
    if not os.path.exists(os.path.join(*path)):
        os.makedirs(os.path.join(*path))

mkdir_if_not_exist(['./train_round2/train_valid'])

In [3]:
df_train = pd.read_csv('./train_round2/Annotations/label.csv', header=None)
df_train.columns = ['image_id', 'class', 'label']
df_train.head()

,image_id,class,label
0,Images/collar_design_labels/b23a3fe060d71bed77...,collar_design_labels,ynnnn
1,Images/collar_design_labels/0ef580b4deabcd9fa4...,collar_design_labels,ynnnn
2,Images/collar_design_labels/87ccc33937821a97ad...,collar_design_labels,nnynn
3,Images/collar_design_labels/26caac7d1f1b36fb9d...,collar_design_labels,ynnnn
4,Images/collar_design_labels/396ab4e7cbc6798100...,collar_design_labels,nynnn


In [4]:
classes = ['collar_design_labels', 'neckline_design_labels', 'skirt_length_labels',
          'sleeve_length_labels', 'neck_design_labels', 'coat_length_labels', 'lapel_design_labels',
          'pant_length_labels']

In [5]:
cur_class = classes[6]
df_load = df_train[(df_train['class'] == cur_class)].copy()
df_load.reset_index(inplace=True)
del df_load['index']

print('{0}: {1}'.format(cur_class, len(df_load)))
df_load.head()

lapel_design_labels: 8876


,image_id,class,label
0,Images/lapel_design_labels/bf1e0ae60afeafeeccf...,lapel_design_labels,nnnny
1,Images/lapel_design_labels/fa185f8c9d166793acf...,lapel_design_labels,ynnnn
2,Images/lapel_design_labels/f5d6d847c3ee7d4fc2e...,lapel_design_labels,nnnny
3,Images/lapel_design_labels/8774f7de05922f0c448...,lapel_design_labels,nnnyn
4,Images/lapel_design_labels/6612436b2c115e7acbc...,lapel_design_labels,ynnnn


In [6]:
task = cur_class
image_path = []
for i in range(len(df_load)):
    image_path.append(('./train_round2/' + df_load['image_id'][i], df_load['label'][i]))

In [7]:
image_path

[('./train_round2/Images/lapel_design_labels/bf1e0ae60afeafeeccf112f924c3892c.jpg',
  'nnnny'),
 ('./train_round2/Images/lapel_design_labels/fa185f8c9d166793acfb8d4725beb603.jpg',
  'ynnnn'),
 ('./train_round2/Images/lapel_design_labels/f5d6d847c3ee7d4fc2eb64f7cb856c96.jpg',
  'nnnny'),
 ('./train_round2/Images/lapel_design_labels/8774f7de05922f0c448ee310f634917d.jpg',
  'nnnyn'),
 ('./train_round2/Images/lapel_design_labels/6612436b2c115e7acbc9d9c7dc729a79.jpg',
  'ynnnn'),
 ('./train_round2/Images/lapel_design_labels/ca9ad9666250256862393193c471c60f.jpg',
  'ynnnn'),
 ('./train_round2/Images/lapel_design_labels/a252e088ff4ebe33d24af549255a38c1.jpg',
  'nynnn'),
 ('./train_round2/Images/lapel_design_labels/799f6f8105377b000c653f469171c24b.jpg',
  'ynnnn'),
 ('./train_round2/Images/lapel_design_labels/7a807b6af858122a2264c6bdd045c18f.jpg',
  'nnynn'),
 ('./train_round2/Images/lapel_design_labels/47a13fd00c9981fe8e25cda3f59c31f0.jpg',
  'nynnn'),
 ('./train_round2/Images/lapel_design_la

In [8]:
n_class = len(df_load['label'][0])
width = 299 # 定义图片大小

In [9]:
mkdir_if_not_exist(['./train_round2/train_valid', task])
mkdir_if_not_exist(['./train_round2/train_valid', task, 'train'])
mkdir_if_not_exist(['./train_round2/train_valid', task, 'val'])
m = len(list(image_path[0][1]))
for mm in range(m):
    mkdir_if_not_exist(['./train_round2/train_valid', task, 'train', str(mm)])
    mkdir_if_not_exist(['./train_round2/train_valid', task, 'val', str(mm)])

## 划分训练集和验证集

In [10]:
import random, shutil

n = len(image_path)
random.seed(1024)
random.shuffle(image_path)
train_count = 0
for path, label in image_path:
    label_index = list(label).index('y')
    if train_count < n * 0.9:
        shutil.copy(path,
                    os.path.join('./train_round2/train_valid', task, 'train', str(label_index)))
    else:
        shutil.copy(path,
                    os.path.join('./train_round2/train_valid', task, 'val', str(label_index)))
    train_count += 1

## 提取特征

In [12]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import *
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import RMSprop
import keras
import gc

Using TensorFlow backend.


In [13]:
cnn_model = InceptionResNetV2(include_top=False, input_shape=(width, width, 3), weights='imagenet')

In [14]:
inputs = Input((width, width, 3))

x = inputs
x = Lambda(preprocess_input, name='preprocessing')(x)
x = cnn_model(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax', name='softmax')(x)

model = Model(inputs, x)

In [15]:
prefix_cls = cur_class.split('_')[0]
prefix_cls

'lapel'

## 划分训练/测试集

In [16]:
# Compile the model
adam = Adam(lr=0.001) 

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
# Set a learning rate annealer
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
#                                             patience=3,
#                                             verbose=1,
#                                             factor=0.1,
#                                             min_lr=0.00001)
epochs = 16
batch_size = 32

## 数据增强

In [17]:
datagen = ImageDataGenerator(
        featurewise_center = False, # set input mean to 0 over the dataset
        samplewise_center = False, # set each sample mean to 0
        featurewise_std_normalization = False, # divide inputs by std of the dataset
        samplewise_std_normalization = False, # divide each input by its std
        zca_whitening = False, # apply ZCA whitening
        rotation_range = 10, # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.07, # randomly zoom image
        width_shift_range = 0.08, # randomly shift images horizontally (fraction of total width)
        height_shift_range = 0.08, # randomly shift images vertivally (fraction of total heigth)
        horizontal_flip = True, # randomly flip images
        vertical_flip = False,
        shear_range = 0.05,
        fill_mode = 'constant',
        cval = 0)

In [18]:
train_generator = datagen.flow_from_directory(
        './train_round2/train_valid/{0}/train'.format(task),
        target_size=(width, width),
        batch_size=32,
        class_mode='categorical')

validation_generator = datagen.flow_from_directory(
        './train_round2/train_valid/{0}/val'.format(task),
        target_size=(width, width),
        batch_size=32,
        class_mode='categorical')

Found 7989 images belonging to 5 classes.
Found 887 images belonging to 5 classes.


- 少量旋转
- 少量偏移
- 水平翻转
- 垂直翻转

In [19]:
int(len(image_path)*0.9)

7988

In [ ]:
prefix_cls = cur_class.split('_')[0]

checkpointer = ModelCheckpoint(filepath='./models/{0}.best0503_InceptionResNetV2.h5'.format(prefix_cls), verbose=1, 
                               save_best_only=True, mode='val_acc')

try:
    # Fit the model
    history = model.fit_generator(train_generator,
                                  epochs=epochs,
                                  validation_data = validation_generator,
                                  #validation_data = (X_valid, y_valid),
                                  verbose=2, 
                                  steps_per_epoch= int(len(image_path)*0.9) // batch_size,
                                  validation_steps= int(len(image_path)*0.9) // batch_size,
                                  callbacks=[EarlyStopping(patience=5), checkpointer])
except KeyboardInterrupt:
    print('KeyboardInterrupt')

Epoch 1/16


In [14]:
cnn_model = InceptionResNetV2(include_top=False, input_shape=(width, width, 3), weights='imagenet')

In [15]:
inputs = Input((width, width, 3))

x = inputs
x = Lambda(preprocess_input, name='preprocessing')(x)
x = cnn_model(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax', name='softmax')(x)

model = Model(inputs, x)

In [ ]:
# Compile the model
adam = Adam(lr=0.0001) 

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
# Set a learning rate annealer
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
#                                             patience=3,
#                                             verbose=1,
#                                             factor=0.1,
#                                             min_lr=0.00001)
epochs = 16
batch_size = 32

In [38]:
model.load_weights('./models/{}.best0503_InceptionResNetV2.h5'.format(prefix_cls)) #加载以前训练好的模型，继续测试

In [40]:
prefix_cls = cur_class.split('_')[0]

checkpointer = ModelCheckpoint(filepath='./models/{0}.best0503_InceptionResNetV2.h5'.format(prefix_cls), verbose=1, 
                               save_best_only=True, mode='val_acc')

try:
    # Fit the model
    history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size, shuffle = True),
                                  epochs=epochs,
                                  #validation_data = vaildgen.flow(X_valid, y_valid,batch_size=batch_size, shuffle = False),
                                  validation_data = (X_valid, y_valid),
                                  verbose=2, 
                                  steps_per_epoch=X_train.shape[0] // batch_size,
                                  validation_steps= X_train.shape[0] // batch_size,
                                  callbacks=[EarlyStopping(patience=5), checkpointer])
except KeyboardInterrupt:
    print('KeyboardInterrupt')

Epoch 1/16
Epoch 00001: val_loss improved from inf to 0.31842, saving model to ./models/pant.best0419_InceptionResNetV2.h5
 - 211s - loss: 0.2456 - acc: 0.9073 - val_loss: 0.3184 - val_acc: 0.8839
Epoch 2/16
Epoch 00002: val_loss did not improve
 - 168s - loss: 0.1771 - acc: 0.9360 - val_loss: 0.3609 - val_acc: 0.8761
Epoch 3/16
Epoch 00003: val_loss did not improve
 - 169s - loss: 0.1360 - acc: 0.9523 - val_loss: 0.3295 - val_acc: 0.8839
Epoch 4/16
KeyboardInterrupt


## 加载模型

In [8]:
df_test = pd.read_csv('./week-rank/Tests/question.csv', header=None)
df_test.columns = ['image_id', 'class', 'x']
del df_test['x']
df_test.head()

,image_id,class
0,Images/collar_design_labels/faad3490a16c7f3d4f...,collar_design_labels
1,Images/collar_design_labels/0b2b4254f35ce3a41a...,collar_design_labels
2,Images/collar_design_labels/7f2be608e06f804dd5...,collar_design_labels
3,Images/collar_design_labels/4b09d4dca80caac42e...,collar_design_labels
4,Images/collar_design_labels/de91f00a05e84d7239...,collar_design_labels


In [9]:
df_load = df_test[(df_test['class'] == cur_class)].copy()
df_load.reset_index(inplace=True)
del df_load['index']

print('{0}: {1}'.format(cur_class, len(df_load)))
df_load.head()

pant_length_labels: 1434


,image_id,class
0,Images/pant_length_labels/e78538758763e84e9700...,pant_length_labels
1,Images/pant_length_labels/1e680886dd3d65882745...,pant_length_labels
2,Images/pant_length_labels/7e25106e7a0f5ac5d26a...,pant_length_labels
3,Images/pant_length_labels/53fa07f57908d258f4ba...,pant_length_labels
4,Images/pant_length_labels/737319237b262a9515a5...,pant_length_labels


In [10]:
'./week-rank/{0}'.format(df_load['image_id'][0])

'./z_rank/Images/pant_length_labels/e78538758763e84e97009dda11754b47.jpg'

In [11]:
n = len(df_load)
X_test = np.zeros((n, width, width, 3), dtype=np.uint8)

for i in tqdm(range(n)):
    X_test[i] = cv2.resize(cv2.imread('./z_rank/{0}'.format(df_load['image_id'][i])), (width, width))


100%|██████████| 1434/1434 [00:08<00:00, 173.38it/s]


In [17]:
model.load_weights('./models/{}.best0502_InceptionResNetV2.h5'.format(prefix_cls))

In [18]:
test_np = model.predict(X_test, batch_size=256)

In [19]:
test_np.shape

(1434, 6)

In [20]:
result = []

for i, row in df_load.iterrows():
    tmp_list = test_np[i]
    tmp_result = ''
    for tmp_ret in tmp_list:
        tmp_result += '{:.4f};'.format(tmp_ret)
        
    result.append(tmp_result[:-1])

df_load['result'] = result
df_load.head()

,image_id,class,result
0,Images/pant_length_labels/e78538758763e84e9700...,pant_length_labels,0.0000;0.0000;0.0000;0.0038;0.1913;0.8049
1,Images/pant_length_labels/1e680886dd3d65882745...,pant_length_labels,0.0057;0.0094;0.7786;0.1921;0.0107;0.0035
2,Images/pant_length_labels/7e25106e7a0f5ac5d26a...,pant_length_labels,0.8354;0.1444;0.0087;0.0020;0.0045;0.0051
3,Images/pant_length_labels/53fa07f57908d258f4ba...,pant_length_labels,0.0003;0.0000;0.0004;0.9891;0.0099;0.0003
4,Images/pant_length_labels/737319237b262a9515a5...,pant_length_labels,0.0000;0.0000;0.0000;0.0004;0.0578;0.9417


In [21]:
df_load.to_csv('./result/{}_0502_InceptionResNetV2.csv'.format('pant'), header=None, index=False)